In [1]:
# Data Imports
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

# Math
import math

# Plot imports
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

# Machine Learning Imports
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

# For evaluating our ML results
from sklearn import metrics

# Dataset Import
import statsmodels.api as sm

C:\Users\ADStudent\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\ADStudent\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
import keras

C:\Users\ADStudent\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
df = sm.datasets.fair.load_pandas().data

In [4]:
df.head()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affairs
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,0.111111
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,3.230769
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1.400000
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,0.727273
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,4.666666


In [5]:
#Create a dummy variable for the women who had affair
def affair_check(x):
    if x != 0:
        return 1
    else:
        return 0

In [6]:
df['Had_Affair'] = df['affairs'].apply(affair_check)

In [7]:
#Create dummies for occupations
occ_dummies = pd.get_dummies(df['occupation'])
hus_occ_dummies = pd.get_dummies(df['occupation_husb'])

In [8]:
occ_dummies.columns = ['occ1','occ2','occ3','occ4','occ5','occ6']
hus_occ_dummies.columns = ['hocc1','hocc2','hocc3','hocc4','hocc5','hocc6']

In [9]:
X = df.drop(['occupation','occupation_husb','Had_Affair'],axis=1)

In [10]:
dummies = pd.concat([occ_dummies,hus_occ_dummies],axis=1)

In [11]:
#merge the control variables into vector X
X = pd.concat([X,dummies],axis=1)

In [12]:
#Drop reference group dummy variables to avoid collinearity problem
X = X.drop(['occ1','hocc1','affairs'],axis=1)
X.head()

,rate_marriage,age,yrs_married,children,religious,educ,occ2,occ3,occ4,occ5,occ6,hocc2,hocc3,hocc4,hocc5,hocc6
0,3.0,32.0,9.0,3.0,3.0,17.0,1,0,0,0,0,0,0,0,1,0
1,3.0,27.0,13.0,3.0,1.0,14.0,0,1,0,0,0,0,0,1,0,0
2,4.0,22.0,2.5,0.0,1.0,16.0,0,1,0,0,0,0,0,0,1,0
3,4.0,37.0,16.5,4.0,3.0,16.0,0,0,0,1,0,0,0,0,1,0
4,5.0,27.0,9.0,1.0,1.0,14.0,0,1,0,0,0,0,0,1,0,0


In [13]:
#Define the dependent variable (in this case binary outcome variable, 1 = Had Affair, 0 = Had no Affair)
Y = df.Had_Affair

In [14]:
#Implement train-test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [15]:
len(X_train)

4456

In [16]:
len(X_test)

1910

In [17]:
X_train.shape

(4456, 16)

In [18]:
mean = X_train.mean(axis=0)
X_train -= mean
std = X_train.std(axis=0)
X_train /= std

X_test -= mean
X_test /= std

In [19]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [20]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [21]:
from keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [22]:
history = model.fit(X_train,
                    y_train,
                    epochs=20)

Epoch 1/20
4456/4456 [==============================] - 1s 178us/step - loss: 0.5717 - acc: 0.7044
Epoch 2/20
4456/4456 [==============================] - 0s 32us/step - loss: 0.5450 - acc: 0.7235
Epoch 3/20
4456/4456 [==============================] - 0s 31us/step - loss: 0.5400 - acc: 0.7244
Epoch 4/20
4456/4456 [==============================] - 0s 32us/step - loss: 0.5357 - acc: 0.7260
Epoch 5/20
4456/4456 [==============================] - 0s 32us/step - loss: 0.5323 - acc: 0.7316
Epoch 6/20
4456/4456 [==============================] - 0s 31us/step - loss: 0.5296 - acc: 0.7303
Epoch 7/20
4456/4456 [==============================] - 0s 36us/step - loss: 0.5283 - acc: 0.7318
Epoch 8/20
4456/4456 [==============================] - 0s 34us/step - loss: 0.5270 - acc: 0.7392
Epoch 9/20
4456/4456 [==============================] - 0s 45us/step - loss: 0.5254 - acc: 0.7365
Epoch 10/20
4456/4456 [==============================] - 0s 38us/step - loss: 0.5236 - acc: 0.7383
Epoch 11/20
4456/4